In [4]:
!pip install tensorflow_hub

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.2 MB/s eta 0:00:00


In [6]:
import json
import pandas as pd
import collections
import numpy as np


import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
import nltk
from nltk import word_tokenize
import nltk
nltk.download('punkt_tab') #for some reason I needed all of these downloaded
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\abiga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abiga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abiga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ModuleNotFoundError: No module named 'tensorflow_datasets'

In [8]:
DIRNAME = 'jeopardy.json'
english_stopwords = set(stopwords.words('english') + list(punctuation) + ['..','...','....','``',"''", '//n'])

In [10]:
jepDict = json.load(open(DIRNAME))

In [12]:
for entry in jepDict:
  values = entry.get('value', '')
  if values:
    cleanVals = values.replace(',','').replace('$', '')
    value = int(cleanVals)
  elif values == 'None':
    value = 0
  entry['value'] = value

In [14]:
def clean_wordlist(jsonList):
  '''
  Takes a list of dictionaries, and provides
  a word list for all the questions and answers, lemmatized wihout stopwwords.
  '''
  corpus = ''
  lemmatizer = WordNetLemmatizer()
  for dictionary in jsonList:
    question = dictionary.get('question').lower()
    answer = dictionary.get('answer').lower()
    corpus += '\n'.join([question, answer])
  wordlist = [lemmatizer.lemmatize(x) for x in word_tokenize(corpus) if x not in english_stopwords]
  return wordlist

In [16]:
jepWords = str(clean_wordlist(jepDict))

In [18]:
hard_text = [entry for entry in jepDict if entry['value'] > 600]
hardWords = set(clean_wordlist(hard_text))

In [20]:
labels = [ 1 if word in hardWords else 0 for word in jepWords]

df = pd.pandas.DataFrame({'text':jepWords, 'hard':labels})

In [ ]:
# Remove rows with missing or NaN values
df = df.dropna(subset=['text', 'hard'])

# Convert all words to strings
df['text'] = df['text'].astype(str)

# Strip whitespace and lowercase (optional but recommended)
df['text'] = df['text'].str.strip().str.lower()

# Remove empty strings
df = df[df['text'] != ""]

# Ensure labels are integers (0/1)
df['hard'] = df['hard'].astype(int)

In [271]:
features = df['text'].to_numpy()
labels = df['hard'].to_numpy()

In [272]:
# Convert everything to string and strip whitespace
features_clean = [str(w).strip() for w in features if str(w).strip() != ""]

labels_clean = [labels[i] for i, w in enumerate(features) if str(w).strip() != ""]

# Sanity check
print("Number of samples:", len(features_clean))
print("First 10 features:", features_clean[:10])
print("First 10 labels:", labels_clean[:10])


Number of samples: 2280404
First 10 features: ["'for", 'last', '8', 'year', 'life', 'galileo', 'house', 'arrest', 'espousing', 'man']
First 10 labels: [np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1)]


In [273]:
train, val, test = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [275]:
def df_to_dataset(dataframe, shuffle=True, batch_size=10000):
    df = dataframe.copy()
    labels = df.pop('hard')
    df = df['text']
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)

Using a model that is already trained that just needs a few tweaks.

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activationn = 'sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.evaluate(train_data)

In [ ]:
model.evaluate(test_data)

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=valid_data)

I honestly do not know if this is actually going to work. Colab RAM started skyrocketing and I could not for the life of me install what was needed on jupyter. I am tired, I have worked on this for 10+ hours today and restarted at hour 8. I watched this video and my sessions would not get past my cleaning section before it crashed. https://www.bing.com/videos/riverview/relatedvideo?q=binary+text+classification+tensorflow+tutorial+for+beginners&&view=riverview&mmscn=mtsc&mid=AE3CDD1455739689F86AAE3CDD1455739689F86A&&aps=1142&FORM=VMSOVR So the model training and splitting is mainly just copied from what I watched, applied to my already cleaned data. She goes into overfitting in the video but I am skipping over that for now. I really liked this video, I wish I knew how it worked out with this data, but I though that she did a great job of explaining things and know a lot more about TensorFlow now.